# 01. GPTQ 베이스라인 — EXAONE-4.0-1.2B

**목표:** 대회 베이스라인(LB ≈ 0.5)을 재현하고, 캘리브레이션 파라미터를 튜닝해서 성능을 개선한다.

**환경:** Google Colab Free (T4 16GB)  
**평가 서버 환경:** L4 GPU, vLLM 0.14.1, Python 3.11

---
### 전략 요약
- 베이스라인: GPTQ W4A16, 캘리브 256샘플, max_len 512 → LB 0.5
- 실험 1: 캘리브 샘플 수 ↑ (512 / 1024) → 정확도 개선 기대
- 실험 2: max_sequence_length ↑ (1024 / 2048) → 더 긴 컨텍스트 캘리브
- 실험 3: MANTA-1M 외 데이터셋 조합

## 0. 환경 확인

In [ ]:
# GPU 확인
import subprocess
result = subprocess.run(['nvidia-smi'], capture_output=True, text=True)
print(result.stdout)

import torch
print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

## 1. 패키지 설치

> ⚠️ 설치 후 **런타임 재시작** 필요. 아래 셀 실행 → 런타임 재시작 → 2번 셀부터 다시 실행

In [ ]:
# llmcompressor: 대회 베이스라인과 동일한 양자화 라이브러리
# compressed-tensors: 평가 서버에도 설치된 버전(0.13.0)과 호환
!pip install -q \
    llmcompressor \
    "compressed-tensors==0.13.0" \
    "transformers>=4.54.0" \
    datasets \
    accelerate \
    sentencepiece

print("\n✅ 설치 완료! 런타임을 재시작하세요 (메뉴 → 런타임 → 런타임 다시 시작)")

## 2. 설정값 (여기서 실험 파라미터 변경)

In [ ]:
# ============================================================
# 실험 설정 — 여기만 바꾸면서 실험
# ============================================================

# 모델
MODEL_ID = "LGAI-EXAONE/EXAONE-4.0-1.2B"   # HuggingFace에서 직접 로드
OUT_DIR  = "./model"                          # 양자화된 모델 저장 위치

# 캘리브레이션 데이터
DATASET_ID    = "LGAI-EXAONE/MANTA-1M"       # EXAONE 공식 데이터셋 (베이스라인과 동일)
DATASET_SPLIT = "train"

# ⬇️ 실험 포인트 1: 샘플 수 늘릴수록 정확도 UP, 시간 UP
# 베이스라인=256 / 권장실험: 512 / 최대권장(T4): 1024
NUM_CALIBRATION_SAMPLES = 512

# ⬇️ 실험 포인트 2: 시퀀스 길이 늘릴수록 캘리브 품질 UP, 메모리 UP
# 베이스라인=512 / 권장실험: 1024 / T4 한계: ~1024
MAX_SEQUENCE_LENGTH = 1024

# 양자화 설정 (베이스라인과 동일 — W4A16: 가중치 4bit, 활성화 16bit)
SCHEME  = "W4A16"
TARGETS = ["Linear"]
IGNORE  = ["embed_tokens", "lm_head"]  # 임베딩/출력 레이어는 양자화 제외

# 제출 파일명
ZIP_NAME = "submit_gptq_w4a16"

print("=" * 50)
print(f"모델        : {MODEL_ID}")
print(f"캘리브 샘플 : {NUM_CALIBRATION_SAMPLES}")
print(f"시퀀스 길이 : {MAX_SEQUENCE_LENGTH}")
print(f"양자화 방식 : {SCHEME}")
print(f"저장 경로   : {OUT_DIR}")
print("=" * 50)

## 3. 모델 & 토크나이저 로드

> T4 16GB에서 1.2B bfloat16 모델은 약 2.5GB → 여유 있음

In [ ]:
import os
import torch
import shutil
from pathlib import Path

from transformers import AutoModelForCausalLM, AutoTokenizer

print("[1/4] 토크나이저 로드 중...")
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_ID,
    trust_remote_code=True,
)
print(f"      vocab size: {tokenizer.vocab_size:,}")

print("[2/4] 모델 로드 중... (최초 실행 시 HuggingFace에서 다운로드, 약 2.5GB)")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    # device_map 미지정 → llmcompressor가 직접 GPU 배치 관리
)

# 모델 구조 간략 확인
total_params = sum(p.numel() for p in model.parameters())
print(f"      총 파라미터: {total_params/1e9:.2f}B")
print(f"      레이어 수  : {model.config.num_hidden_layers}")
print(f"      Attention  : {model.config.num_attention_heads} heads / {model.config.num_key_value_heads} KV heads (GQA)")
print("\n✅ 모델 로드 완료")

## 4. 캘리브레이션 데이터 준비

In [ ]:
from datasets import load_dataset

print(f"[3/4] 캘리브레이션 데이터 로드 중: {DATASET_ID} ({NUM_CALIBRATION_SAMPLES}샘플)")

ds = load_dataset(
    DATASET_ID,
    split=f"{DATASET_SPLIT}[:{NUM_CALIBRATION_SAMPLES}]",
)

def preprocess(example):
    """대화 형식을 EXAONE chat template에 맞게 변환"""
    return {
        "text": tokenizer.apply_chat_template(
            example["conversations"],
            add_generation_prompt=True,
            tokenize=False,
        )
    }

ds = ds.map(preprocess)

# 샘플 미리보기
print("\n--- 샘플 데이터 미리보기 (첫 번째 항목 앞 200자) ---")
print(ds[0]["text"][:200])
print("...")
print(f"\n✅ 데이터 준비 완료: {len(ds)}개 샘플")

## 5. GPTQ 양자화 실행

> T4에서 512샘플 기준 약 15~25분 소요

In [ ]:
from llmcompressor import oneshot
from llmcompressor.modifiers.quantization import GPTQModifier
import time

print(f"[4/4] GPTQ 양자화 시작")
print(f"      Scheme : {SCHEME}")
print(f"      Samples: {NUM_CALIBRATION_SAMPLES}")
print(f"      MaxLen : {MAX_SEQUENCE_LENGTH}")
print("-" * 40)

recipe = [
    GPTQModifier(
        scheme=SCHEME,
        targets=TARGETS,
        ignore=IGNORE,
    )
]

start_time = time.time()

oneshot(
    model=model,
    dataset=ds,
    recipe=recipe,
    max_seq_length=MAX_SEQUENCE_LENGTH,
    num_calibration_samples=NUM_CALIBRATION_SAMPLES,
)

elapsed = time.time() - start_time
print(f"\n✅ GPTQ 완료! 소요 시간: {elapsed/60:.1f}분")

## 6. 모델 저장 & 제출 파일 생성

In [ ]:
import os
import shutil

# 저장
os.makedirs(OUT_DIR, exist_ok=True)
print(f"모델 저장 중: {OUT_DIR}")

model.save_pretrained(OUT_DIR, save_compressed=True)
tokenizer.save_pretrained(OUT_DIR)

# 저장된 파일 확인
print("\n저장된 파일:")
for f in sorted(Path(OUT_DIR).iterdir()):
    size_mb = f.stat().st_size / 1024**2
    print(f"  {f.name:40s} {size_mb:8.1f} MB")

total_size = sum(f.stat().st_size for f in Path(OUT_DIR).iterdir()) / 1024**3
print(f"\n총 크기: {total_size:.2f} GB (제출 한도: 압축 후 10GB / 압축 전 32GB)")

print("\n✅ 저장 완료")

In [ ]:
# 제출용 zip 생성
print(f"제출 파일 생성 중: {ZIP_NAME}.zip")

shutil.make_archive(
    base_name=ZIP_NAME,
    format="zip",
    root_dir=".",
    base_dir=OUT_DIR,
)

zip_size = Path(f"{ZIP_NAME}.zip").stat().st_size / 1024**3
print(f"\n✅ {ZIP_NAME}.zip 생성 완료")
print(f"   압축 파일 크기: {zip_size:.2f} GB")
print("   → Dacon 제출 페이지에 이 zip 파일을 업로드하세요")

## 7. 빠른 sanity check (양자화 결과 확인)

제출 전에 양자화된 모델이 정상 추론되는지 확인

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

print("양자화된 모델 로드 중...")
model_check = AutoModelForCausalLM.from_pretrained(
    OUT_DIR,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True,
)
tok_check = AutoTokenizer.from_pretrained(OUT_DIR, trust_remote_code=True)

# 간단한 추론 테스트
prompt = "대한민국의 수도는"
messages = [{"role": "user", "content": prompt}]
inputs = tok_check.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model_check.device)

with torch.no_grad():
    outputs = model_check.generate(inputs, max_new_tokens=50, do_sample=False)

response = tok_check.decode(outputs[0][inputs.shape[-1]:], skip_special_tokens=True)
print(f"\n테스트 프롬프트: {prompt}")
print(f"모델 응답: {response}")
print("\n✅ Sanity check 완료! 모델이 정상 작동합니다.")

---
## 📊 실험 결과 기록

아래 표에 실험할 때마다 결과를 기록해두세요:

| 실험 | SCHEME | 샘플 수 | MaxLen | 데이터셋 | LB Score | 비고 |
|------|--------|---------|--------|----------|----------|------|
| 베이스라인 | W4A16 | 256 | 512 | MANTA-1M | 0.5 | 대회 제공 베이스라인 |
| 실험1 | W4A16 | 512 | 1024 | MANTA-1M | - | 이 노트북 기본 설정 |
| 실험2 | W4A16 | 1024 | 1024 | MANTA-1M | - | 샘플 수 2배 |

---
## ⏭️ 다음 단계

- GPTQ 결과를 보고 점수가 낮으면 → `02_fp8_quantization.ipynb` 로 FP8 시도 (L4에서 하드웨어 가속 기대)
- 점수가 잘 나오면 → `04_qlora_sft.ipynb` 로 성능 추가 회복